In [15]:
import pandas as pd 
import yfinance as yf 
from scipy.stats import spearmanr
import networkx as nx 
import numpy as np 

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

#read wikipedia S&P500 table into PD dataframe 
df_sp500 = pd.read_html(url)[0][['Symbol', 'Security', 'GICS Sector']] 
#ticker DF 
sp500_tickers = df_sp500['Symbol'].tolist() 

#read changes table into DF 
#sp500_changes = pd.read_html(url)[1]
#problematic 
# drop rows where at least one element missing
#changes = df_changes[['Date', 'Added', 'Removed']].dropna() 
#filter out reasons column 
#changes = changes[~changes['Date'].str.contains('Reason')]
#filter out note column 
#changes = changes[~changes['Date'].str.contains('Note')] 

#leaving these fuckers out (Berkshire Hathaway - 2010, Brown-Forman - 1982, AirBNB - 2023)
exclude = ['BRK.B', 'BF.B', 'ABNB', 'CARR', 'CEG', 'CTVA', 'DOW', 'FOXA', 'FOX', 'GEHC', 'KVUE', 'OTIS', 'UBER', 'VLTO'] 
sp500_tickers = [ticker for ticker in sp500_tickers if ticker not in exclude]

#dict for individual stock DF storage 
stockdataframes = {} 

start_dte = "2019-01-01"
end_dte = "2023-12-31"

#ADD INCLUSION AND EXCLUSION DATE AND FILTER BY THAT FOR NEWER STOCKS AND REMOVED ONES 

#iterate through all tickers+ get historical data 
for ticker in sp500_tickers: 
    #added = row['Added'] 
    #removed = row['Removed'] 
    
    try:
        stock = yf.Ticker(ticker) 
        stock_df = stock.history(start=start_dte, end=end_dte)[['Close']]
        
        #debug 
        #print(f"Debug - Ticker: {ticker}, DataFrame shape: {stock_df.shape}")
        
        #merge with wiki data - check if not empty first
        if 'Close' in stock_df.columns and not stock_df.empty: 
            stock_df['Symbol'] = ticker
            stock_df['Security'] = df_sp500[df_sp500['Symbol'] == ticker]['Security'].iloc[0]
            stock_df['GICS Sector'] = df_sp500[df_sp500['Symbol'] == ticker]['GICS Sector'].iloc[0]
            #merged_stock_df = pd.merge(stock_df, df_sp500[df_sp500['Symbol'] == ticker], left_index=True, right_index=True)
        
            #store merged df in dictionary 
            stockdataframes[ticker] = stock_df #used o be merged_stock_dataframe
        
            #print(f"Data fetched for {ticker}")
        else:
            print(f"Failed to fetch data for {ticker}: {e}")
    except Exception as e: 
        print(f"Failed to fetch data for {ticker}: {e}")

#merge all stock dfs into one 
full_df = pd.concat(stockdataframes.values(), keys=stockdataframes.keys())

full_df = full_df.reset_index(level=0) 

#ptiny 
print(full_df.head(450)) 
#display(full_df) 
#full_df.to_csv('SP500.csv', index=True)

#resample monthly 
monthly_df = full_df.resample('M').last()

#daily price changes 
pricechgs_df = monthly_df.pivot(columns='Symbol', values='Close').pct_change().dropna() 
print("Monthly Returns:") 
print(pricechgs_df.head(350)) #its ony getting data from ZTS. why? idk 
print("NaN or Inf Returns Checking :(") 
print(pricechgs_df.isnull().sum())
print(pricechgs_df.isin([np.inf, -np.inf]).sum())

#spearman correlation matrix https://www.guru99.com/r-pearson-spearman-correlation.html#:~:text=between%20the%20rank.-,Spearman's%20rank%20correlation%2C%20%2C%20is%20always%20between%20%2D1%20and%201,the%20pearson%20or%20spearman%20method.
#corr_matrix, _ = spearmanr(pricechgs_df, axis=1)
corr_matrix = pricechgs_df.corr(method='spearman') 

#print correlation matrix
print("Spearman Matrix:") 
#print(pd.DataFrame(corr_matrix, index=sp500_tickers, columns=sp500_tickers)) 
print(corr_matrix) 

                          level_0       Close Symbol Security  GICS Sector
Date                                                                      
2019-01-02 00:00:00-05:00     MMM  156.153748    MMM       3M  Industrials
2019-01-03 00:00:00-05:00     MMM  150.273956    MMM       3M  Industrials
2019-01-04 00:00:00-05:00     MMM  156.456329    MMM       3M  Industrials
2019-01-07 00:00:00-05:00     MMM  156.096497    MMM       3M  Industrials
2019-01-08 00:00:00-05:00     MMM  156.750702    MMM       3M  Industrials
...                           ...         ...    ...      ...          ...
2020-10-07 00:00:00-04:00     MMM  144.726944    MMM       3M  Industrials
2020-10-08 00:00:00-04:00     MMM  145.787445    MMM       3M  Industrials
2020-10-09 00:00:00-04:00     MMM  147.169601    MMM       3M  Industrials
2020-10-12 00:00:00-04:00     MMM  146.604568    MMM       3M  Industrials
2020-10-13 00:00:00-04:00     MMM  144.848633    MMM       3M  Industrials

[450 rows x 5 columns]
M

2

In [ ]:
#REF 
#import pandas as pd 
#import yfinance as yf 

#url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

#read tables into PD dataframe 
#dfs = pd.read_html(url) 

#get first table (index 0) + first 3 cols 
#df = dfs[0].iloc[:, :3] 
#print(df) 